## 1. 모델링

사전설정 (모델 훈련과 예측 모두 공통)

In [ ]:
import os

model_save_path = "saved_model1"
# cols = ["KOSPI", "CR", "NASDAQ", "DOW", "NIKKEI", "SHANGHAI", "INDI", "FOREIGN", "ORG"]
cols = ["KOSPI", "CR", "NASDAQ", "DOW", "NIKKEI", "SHANGHAI"]
len_x_ARMA = 10
len_y_nextday = 20
scale_method = "norm"

# 패키지 캐시 모두 지우기 (tf 등 이전에 남은 가비지가 영향을 미칠 수 있음)
for dir_ in ["/Users/hrdkdh/Documents/GitHub/predict/", "/Users/hrdkdh/venv/"]:
    os.system("find {} | grep -E '(__pycache__|\.pyc$|\pyo$)' | xargs rm -rf".format(dir_))

1-1) 데이터 크롤링

In [ ]:
from kospi_predict import Crawler

crawler = Crawler(crawl_page_max=50, perPage=100)
# crawler.crawlData(cols, save=True)
crawler.loadFromSavedFile(cols)
df_crawled = crawler.removeNan()
df_crawled

1-2. 데이터 전처리

In [ ]:
from kospi_predict import DataPreprocessor

dpp = DataPreprocessor(df_crawled, cols, scale_method, model_save_path)
# dpp.removeOutlier()
dpp.sortByDate()
dpp.scalingForModeling()
dpp.makeTargetXs(len_x_ARMA)
dpp.makeTargetYs(len_y_nextday)
dpp.cutoffData(len_x_ARMA, len_y_nextday)
# dpp.cutoffData(1000, len_y_nextday)
dpp.splitData()
dpp.df_cutoff

1-3) 모델 훈련하고 생성

In [ ]:
from kospi_predict import ModelMaker

maker = ModelMaker(dpp.y_list, dpp.df_train, dpp.df_test, "relu", model_save_path)
maker.makeModel(EPOCHS=450, history_plot_cutoff=30, learning_rate=0.0008)

In [ ]:
maker.saveModelByCheckpoint("0285")
maker.validateModel(y_no=0)

In [ ]:
print(maker.test_mae)
print(maker.test_mse)

## 2. 최근 데이터로 20일 간의 KOSPI 지수 예측

2-1) 데이터 수집

In [ ]:
from kospi_predict import Crawler

#예측을 위해서는 현재기준 이전 10일간의 데이터만 있으면 됨 (len_x_ARMA가 10일때)
crawler_pred = Crawler(crawl_page_max=1, perPage=50) #이전 추세도 볼겸 넉넉히 수집
crawler_pred.crawlData(cols)
df_crawled_pred = crawler_pred.removeNan()
df_crawled_pred

2-2) 데이터 전처리

In [ ]:
from kospi_predict import DataPreprocessor

#아래는 예측을 위한 전처리
dpp_pred = DataPreprocessor(df_crawled_pred, cols, scale_method, model_save_path)
dpp_pred.sortByDate()
dpp_pred.scalingForPredict()
dpp_pred.makeTargetXs(len_x_ARMA)
dpp_pred.cutoffData(len_x_ARMA, 0)

2-3) 예측

In [ ]:
from kospi_predict import Predictor

#2.18 기준으로 20일 뒤까지 예측하고 시각화

predictor = Predictor(dpp_pred.df, scaled=True, scale_method = scale_method, model_save_path=model_save_path)
predictor.predict()
predictor.showPredictionPlot()

In [ ]:
len(predictor.x_cols)
predictor.saveModelToJS()